In [ ]:
import cupy as cp
import datashader as ds
import datashader.transfer_functions as tf
import pandas as pd
import xarray as xr
import numpy as np
from PIL import Image
import os
import imageio.v2 as imageio
from matplotlib import cm
from tqdm import trange 
import subprocess
import time
import threading
import queue     


# Set up cuda kernel for mandelbrot, and compute via cupy

In [ ]:
mandelbrot_kernel = cp.RawKernel(r'''
extern "C" __global__
void mandelbrot(float* output, float x_min, float x_max, float y_min, float y_max, 
                int width, int height, int max_iter) {
    int tidx = blockDim.x * blockIdx.x + threadIdx.x;
    int tidy = blockDim.y * blockIdx.y + threadIdx.y;

    if (tidx >= width || tidy >= height) return;

    int idx = tidy * width + tidx;

    float x0 = x_min + tidx * (x_max - x_min) / width;
    float y0 = y_min + tidy * (y_max - y_min) / height;

    float x = 0.0;
    float y = 0.0;
    int iteration = 0;

    while (x*x + y*y <= 4.0 && iteration < max_iter) {
        float xtemp = x*x - y*y + x0;
        y = 2*x*y + y0;
        x = xtemp;
        iteration++;
    }

    output[idx] = iteration;
}
''', 'mandelbrot')


def compute_mandelbrot(width, height, max_iter, x_range, y_range):
    output = cp.zeros((height, width), dtype=cp.float32)

    block = (16, 16)
    grid = ((width + block[0] - 1) // block[0],
            (height + block[1] - 1) // block[1])

    mandelbrot_kernel(grid, block,
        (output,
         cp.float32(x_range[0]), cp.float32(x_range[1]),
         cp.float32(y_range[0]), cp.float32(y_range[1]),
         cp.int32(width), cp.int32(height), cp.int32(max_iter))
    )
    return cp.asnumpy(output) 



# Set up visuliazation methods (using normaliziation, raster xaraay, and datashader)

In [3]:
def normalize_for_coloring(data, max_iter):
    #map values to [0, 1], leave max_iter (interior) at 1.0 for white
    norm = np.clip(data / max_iter, 0, 0.7)
    return norm


def render_gradient(data, x_range, y_range):
    height, width = data.shape
    da = xr.DataArray(data, dims=["y", "x"],
                      coords={"x": np.linspace(x_range[0], x_range[1], width),
                              "y": np.linspace(y_range[0], y_range[1], height)})

    cvs = ds.Canvas(plot_width=width, plot_height=height,
                    x_range=x_range, y_range=y_range)


    cmap = ["black", "yellow", "black", "black", "yellow", "black"]
    img = tf.shade(cvs.raster(da), cmap=cmap, how='linear')
    return img


In [4]:


WIDTH = 1024
HEIGHT = 1024
MAX_ITER = 1000
X_RANGE = (-2, 0.5)  
Y_RANGE = (-1.25, 1.25)

mandelbrot_data = compute_mandelbrot(WIDTH, HEIGHT, MAX_ITER, X_RANGE, Y_RANGE)
normalized_data = normalize_for_coloring(mandelbrot_data, MAX_ITER)
img = render_gradient(normalized_data, X_RANGE, Y_RANGE)
img.to_pil().show()


CUDARuntimeError: cudaErrorIllegalAddress: an illegal memory access was encountered

# Create zoomed video frames

In [ ]:
mandelbrot_kernel = cp.RawKernel(r'''
extern "C" __global__
void mandelbrot(double* output, double x_min, double x_max, double y_min, double y_max,
                int width, int height, int max_iter) {
    int x_idx = blockDim.x * blockIdx.x + threadIdx.x;
    int y_idx = blockDim.y * blockIdx.y + threadIdx.y;

    if (x_idx >= width || y_idx >= height) return;

    int idx = y_idx * width + x_idx;

    // Map pixel coordinates to complex plane
    double x0 = x_min + x_idx * (x_max - x_min) / width;
    double y0 = y_min + y_idx * (y_max - y_min) / height;

    double x = 0.0;
    double y = 0.0;
    double x2 = 0.0;
    double y2 = 0.0;

    int i = 0;
    // Iterate the Mandelbrot equation
    while (x2 + y2 <= 4.0 && i < max_iter) {
        y = 2.0 * x * y + y0;
        x = x2 - y2 + x0;
        x2 = x * x;
        y2 = y * y;
        i++;
    }

    // Store the number of iterations
    output[idx] = (double)i;
}
''', 'mandelbrot')

colorize_kernel = cp.RawKernel(r'''
extern "C" __global__
void colorize_mandelbrot(const double* mandel_data, unsigned char* rgb_output,
                         int width, int height, int max_iter,
                         const float* lut, int lut_size) {
    int x_idx = blockDim.x * blockIdx.x + threadIdx.x;
    int y_idx = blockDim.y * blockIdx.y + threadIdx.y;

    if (x_idx >= width || y_idx >= height) return;

    int idx = y_idx * width + x_idx;       // Index for input mandel_data
    int rgb_idx = idx * 3;                 // Index for output rgb_output (R channel)

    double iter_count_d = mandel_data[idx];
    int iter_count = (int)iter_count_d;

    float norm;
if (iter_count == max_iter) {
    // Inside the set → keep black
    norm = 0.0f;
} else {
    // Outside the set → normalize with log and gamma
    float log_iter = logf(1.0f + (float)iter_count);
    float log_max = logf(1.0f + (float)(max_iter));
    norm = log_iter / log_max;
    norm = powf(norm, 0.8f);  // Gamma correction
    norm = fmaxf(0.0f, fminf(norm, 1.0f));  // Clamp to [0, 1]
}


    // Map normalized value to LUT index
    int lut_idx = (int)(norm * (float)(lut_size - 1));
    lut_idx = max(0, min(lut_idx, lut_size - 1)); // Clamp index to valid range

    // Get RGB values from LUT
    int lut_base_idx = lut_idx * 3;
    rgb_output[rgb_idx + 0] = (unsigned char)(lut[lut_base_idx + 0] * 255.0f); // R
    rgb_output[rgb_idx + 1] = (unsigned char)(lut[lut_base_idx + 1] * 255.0f); // G
    rgb_output[rgb_idx + 2] = (unsigned char)(lut[lut_base_idx + 2] * 255.0f); // B

     // Override: Color points inside the set explicitly black
    if (iter_count == max_iter) {
         rgb_output[rgb_idx + 0] = 0; // R
         rgb_output[rgb_idx + 1] = 0; // G
         rgb_output[rgb_idx + 2] = 0; // B
    }
}
''', 'colorize_mandelbrot')

def compute_mandelbrot(width, height, max_iter, x_range, y_range, stream=None):
    """Computes Mandelbrot iterations on GPU, returns CuPy array."""
    #malloc
    output = cp.zeros((height, width), dtype=cp.float64)
    #block and grid sizes
    block = (32, 32) # 32*32 = 1024 threads/block
    grid = ((width + block[0] - 1) // block[0],
            (height + block[1] - 1) // block[1])
    if grid[0] <= 0 or grid[1] <= 0:
            raise ValueError(f"Calculated grid dimensions are invalid: {grid}")

    #launch Mandelbrot kernel
    mandelbrot_kernel(grid, block, (
        output,
        cp.float64(x_range[0]), cp.float64(x_range[1]),
        cp.float64(y_range[0]), cp.float64(y_range[1]),
        cp.int32(width), cp.int32(height), cp.int32(max_iter)
    ), stream=stream) 

    return output

In [6]:
def create_gpu_lut(cmap, lut_size=8192):
    """Creates a cmap lookup table (LUT) on GPU"""
    try:
        if isinstance(cmap, str):
            cmap_obj = cm.get_cmap(cmap)
        else:
            cmap_obj = cmap  #
    except Exception:
        cmap_obj = cm.get_cmap("inferno")

    #gen colors on CPU (range [0, 1])
    cpu_lut = cmap_obj(np.linspace(0, 1, lut_size))[:, :3]  # RGB only
    gpu_lut = cp.array(cpu_lut, dtype=cp.float32)
    return gpu_lut

def save_worker(q, pbar=None):
    """Pulls tasks (frame_path, image_data) from queue and saves images."""
    while True:
        item = q.get()
        if item is None: 
            break
        frame_path, image_data = item
        try:
            imageio.imwrite(frame_path, image_data, compression=3)
        except Exception as e:
            print(f"Error saving {frame_path}: {e}")
        finally:
            q.task_done() 
            if pbar:
                pbar.update(1) #progress bar update


def generate_zoom_frames_async_save(center, 
                                    scale_start, 
                                    scale_end, 
                                    num_frames, 
                                    width, 
                                    height, 
                                    max_iter, 
                                    output_dir, 
                                    batch_size=8, 
                                    cmap_name="inferno", 
                                    lut_size=1024, 
                                    num_saver_threads=4):
    
    """Generates frames, using background threads for saving."""
    #general setup
    os.makedirs(output_dir, exist_ok=True)
    zooms = np.logspace(np.log10(scale_start), np.log10(scale_end), num=num_frames)
    gpu_lut = create_gpu_lut(cmap_name, lut_size)
    #setup for kernel
    # init block
    block_colorize = (32, 32)
    grid_colorize = ((width + block_colorize[0] - 1) // block_colorize[0],
                     (height + block_colorize[1] - 1) // block_colorize[1])

    #setup for saving
    save_queue = queue.Queue(maxsize=batch_size * 2)
    threads = []
    save_pbar = trange(num_frames, desc="Saving Frames ", unit="frame", leave=True)
    for _ in range(num_saver_threads):
        thread = threading.Thread(target=save_worker, args=(save_queue, save_pbar), daemon=True)
        thread.start()
        threads.append(thread)

    #malloc
    mandel_gpu_batch = [None] * batch_size
    rgb_gpu_batch = [None] * batch_size
    events = [cp.cuda.Event() for _ in range(batch_size)] 

    #show progress
    gen_pbar = trange(0, num_frames, batch_size, desc="Launching Batches", unit="batch", leave=True)
    for i in gen_pbar:
        streams = [cp.cuda.Stream() for _ in range(batch_size)] #streams per batch
        active_streams_in_batch = 0

        #launc kernel batch
        for j in range(batch_size):
            frame_idx = i + j
            if frame_idx >= num_frames: break
            active_streams_in_batch += 1

            zoom = zooms[frame_idx]
            half_width = zoom
            half_height = zoom * (height / width)
            x_range = (center[0] - half_width, center[0] + half_width)
            y_range = (center[1] - half_height, center[1] + half_height)

            with streams[j]:
                # a) Compute Mandelbrot iterations
                mandel_gpu_batch[j] = compute_mandelbrot(width, height, max_iter, x_range, y_range, stream=streams[j])
                # b) Allocate buffer for colorized output on GPU
                rgb_gpu_batch[j] = cp.empty((height, width, 3), dtype=cp.uint8)
                # c) Launch Colorize Kernel
                colorize_kernel(grid_colorize, block_colorize, (
                    mandel_gpu_batch[j], rgb_gpu_batch[j],
                    cp.int32(width), cp.int32(height), cp.int32(max_iter),
                    gpu_lut, cp.int32(lut_size)
                ))
                #record an event after colorize kernel finishes on this stream
                events[j].record(streams[j])

        #wait for GPU, transfer, and queue for Saving

        for j in range(active_streams_in_batch):
            frame_idx = i + j 
            #wait specifically for this frame kernels (Mandelbrot + Colorize) to finish
            events[j].synchronize() 

            #transfer results 
            #for max performance, could explore async transfer with pinned memory
            rgb_cpu_data = cp.asnumpy(rgb_gpu_batch[j])
            frame_path = os.path.join(output_dir, f"frame_{frame_idx:04d}.png")
            save_queue.put((frame_path, rgb_cpu_data.copy()))
            #clear GPU buffers 
            mandel_gpu_batch[j] = None
            rgb_gpu_batch[j] = None

        # Cleanup streams for the completed batch
        del streams


    #finalize saving tasks
    save_queue.join() 
    save_pbar.close() 
    for _ in range(num_saver_threads):
        save_queue.put(None)

    for t in threads:
        t.join()

    #cleanuppgu resources
    del gpu_lut
    del mandel_gpu_batch 
    del rgb_gpu_batch 
    del events
    cp.get_default_memory_pool().free_all_blocks()
    cp.get_default_pinned_memory_pool().free_all_blocks()
    print("GPU cleanup complete.")


def compile_video_ffmpeg_direct(frame_dir, output_path, fps=30):
    """Compiles video using FFmpeg directly via subprocess."""
    print(f"\nCompiling video from frames in '{frame_dir}'...")
    ffmpeg_path = os.environ.get("IMAGEIO_FFMPEG_EXE", "ffmpeg") 
    first_frame_path = os.path.join(frame_dir, 'frame_0000.png')
    #help with AI
    cmd = [
        ffmpeg_path,
        '-y',                                     # Overwrite output file if it exists
        '-framerate', str(fps),                   # Input frame rate
        '-start_number', '0',                     # Ensure numbering starts from 0000
        '-i', os.path.join(frame_dir, 'frame_%04d.png'), # Input image sequence pattern
        '-c:v', 'libx264',                        # Video codec (H.264)
        '-preset', 'fast',                       # Encoding speed/compression trade-off
        '-crf', '18',                             # Constant Rate Factor (quality)
        '-pix_fmt', 'yuv420p',                    # Pixel format for compatibility
        output_path                               # Output file path
    ]

    print(f"Running FFmpeg command: {' '.join(cmd)}")
    try:
        process = subprocess.run(cmd, check=True, capture_output=True, text=True)
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error during FFmpeg compilation!")
        print(f"Command: {e.cmd}")
        print(f"Return Code: {e.returncode}")
        print(f"FFmpeg Output (stderr):\n{e.stderr}")
        print(f"FFmpeg Output (stdout):\n{e.stdout}")
        return False
    except FileNotFoundError:
        print(f"Error: FFmpeg executable not found at '{ffmpeg_path}'.")
        print("Please ensure FFmpeg is installed and the path is correct (set IMAGEIO_FFMPEG_EXE environment variable if needed).")
        return False



In [7]:
from matplotlib.colors import LinearSegmentedColormap
cyberpunk_cmap = LinearSegmentedColormap.from_list("cyberpunk", [
    (0.0, "#000000"),  
    (0.2, "#210535"),  
    (0.5, "#540d6e"),  
    (0.75, "#08ffc8"), 
    (1.0, "#00ff9f")   
])

In [ ]:
#setup run
CENTER = (-0.743643887037151, 0.13182590420533)  
SCALE_START = 1.5
SCALE_END = 1.5e-15  
MAX_ITER = 15_000  
FPS = 60
FRAMES = 600
WIDTH = 2560
HEIGHT = 1440
BATCH_SIZE = 64
NUM_SAVER_THREADS = 32 


CMAP_NAME = cyberpunk_cmap
LUT_SIZE = 8192
frame_dir = OUT_DIR = f"mandelbrot_async_{WIDTH}x{HEIGHT}_{MAX_ITER}iter"
VIDEO_FILE = f"mandelbrot_zoom_async_{WIDTH}x{HEIGHT}_{MAX_ITER}iter_{FPS}fps.mp4"


script_start_time = time.time()
generate_zoom_frames_async_save(
    CENTER, SCALE_START, SCALE_END, FRAMES,
    WIDTH, HEIGHT, MAX_ITER, OUT_DIR,
    batch_size=BATCH_SIZE, cmap_name=CMAP_NAME, lut_size=LUT_SIZE,
    num_saver_threads=NUM_SAVER_THREADS
)

generation_time = time.time() - script_start_time
print(f"\nFrame generation and saving finished in {generation_time:.2f} seconds.")

#compule vid
output_video_file = f"mandelbrot_video_wsl_{int(time.time())}.mp4"
ffmpeg_path = r"/mnt/c/Users/Ruben/GPUcodig/ffmpeg-master-latest-win64-gpl/bin/ffmpeg.exe" 
output_video_file = os.path.join(".", output_video_file)
cmd = [
    ffmpeg_path,
    "-y",
    "-framerate", str(FPS),
    "-start_number", "0",
    "-i", os.path.join(frame_dir, "frame_%04d.png"),
    "-c:v", "libx264",
    "-preset", "fast",
    "-crf", "18",
    "-pix_fmt", "yuv420p",
    output_video_file
]


try:
    subprocess.run(cmd, check=True)
    print(f"\nVideo successfully saved to: {output_video_file}")
except subprocess.CalledProcessError as e:
    print(f"\nOHNONONONO FFmpeg failed with return code {e.returncode}")
    print("STDERR:", e.stderr or "(none)")
except Exception as e:
    print(f"Yikes Unexpected error: {e}")

